<a href="https://colab.research.google.com/github/DanielHevdeli/hafifot-tiug/blob/main/BERT_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/DanielHevdeli/hafifot-tiug.git

Cloning into 'hafifot-tiug'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 79 (delta 31), reused 33 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 7.56 MiB | 7.39 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
import pandas as pd

In [3]:
posts_df = pd.read_csv("./hafifot-tiug/data/split_data/present.csv")

In [4]:
print(f"Num of posts: {len(posts_df)}")

Num of posts: 4000


In [5]:
posts_df.head(2)

,question_id,length,date,text
0,114678,903,2016-04-10 20:05:00,"שלום , אני מאוד מקווה שתוכלו לעזור לי אני לא י..."
1,116349,888,2016-04-24 19:56:00,היי כולם \nיש לי בעיה הקשורה לתספורת שלי. \nאמ...


In [6]:
labels_df = pd.read_csv("./hafifot-tiug/data/labels/present/dummy-gpt-4_labels.csv")

In [7]:
print(f"Num of labels: {len(labels_df)}")

Num of labels: 4000


In [8]:
labels_df.head(2)

,question_id,label
0,114678,suicidal
1,116349,non-suicidal


In [9]:
merged_df = pd.merge(posts_df[['question_id', 'text']], labels_df, on='question_id')
posts_with_labels_df = merged_df[['question_id', 'text', 'label']]
display(posts_with_labels_df.head())

,question_id,text,label
0,114678,"שלום , אני מאוד מקווה שתוכלו לעזור לי אני לא י...",suicidal
1,116349,היי כולם \nיש לי בעיה הקשורה לתספורת שלי. \nאמ...,non-suicidal
2,108590,הגעתי לאתר הזה במקרה אחרי שקראתי טיפה מפוסטים ...,non-suicidal
3,100709,"בערך מגיל 17 אני סובלת מחרדה, כאשר חלה החרפה ק...",suicidal
4,120766,"אני תוהה כבר כמה שנים מה ללמוד בעתיד, איזה מקצ...",non-suicidal


In [10]:
print(f"posts_with_labels_df len: {len(posts_with_labels_df)}")

posts_with_labels_df len: 4000


# Create a BERT Classifier

In [11]:
import torch
from torch import nn
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
from torchsummary import summary
from datasets import Dataset, DatasetDict
import numpy as np
import matplotlib.pyplot as plt

In [36]:
label2id = {"non-suicidal": 0, "suicidal": 1}
id2label = {0: "non-suicidal", 1: "suicidal"}
df = posts_with_labels_df.copy()
df["label_id"] = df["label"].map(label2id)


In [37]:
df.head()

,question_id,text,label,label_id
0,114678,"שלום , אני מאוד מקווה שתוכלו לעזור לי אני לא י...",suicidal,1
1,116349,היי כולם \nיש לי בעיה הקשורה לתספורת שלי. \nאמ...,non-suicidal,0
2,108590,הגעתי לאתר הזה במקרה אחרי שקראתי טיפה מפוסטים ...,non-suicidal,0
3,100709,"בערך מגיל 17 אני סובלת מחרדה, כאשר חלה החרפה ק...",suicidal,1
4,120766,"אני תוהה כבר כמה שנים מה ללמוד בעתיד, איזה מקצ...",non-suicidal,0


In [38]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label_id"], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df["label_id"], random_state=42)

In [39]:
train_ds = Dataset.from_pandas(train_df[["text", "label_id"]])
val_ds   = Dataset.from_pandas(val_df[["text", "label_id"]])
test_ds  = Dataset.from_pandas(test_df[["text", "label_id"]])

ds = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})

In [40]:
MODEL_NAME = "onlplab/alephbert-base"
MAX_LEN = 256
BATCH_SIZE = 16
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [44]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, max_length=MAX_LEN)

ds = ds.map(tokenize, batched=True)

Map:   0%|          | 0/2560 [00:00<?, ? examples/s]

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [46]:
ds = ds.remove_columns(["text"])
ds.set_format(type="torch")

In [47]:
ds

DatasetDict({
    train: Dataset({
        features: ['label_id', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2560
    })
    validation: Dataset({
        features: ['label_id', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 640
    })
    test: Dataset({
        features: ['label_id', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 800
    })
})